<a href="https://colab.research.google.com/github/yalopez84/Interpretable_method/blob/master/Interpretable_Triple_Prediction_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
!pip install sentence_transformers
!pip install umap-learn 
!pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [109]:
from sentence_transformers import SentenceTransformer
import os
import csv
import sys
import random
from tqdm import tqdm, trange
import logging
import pdb
import umap
import hdbscan
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbs
from torch.utils.tensorboard import SummaryWriter
writter = SummaryWriter(log_dir='tensorboard_embedding_custom2')
logger = logging.getLogger(__name__)

In [110]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
pathFile="/content/drive/MyDrive/Almacen_phd/FB13/"
modelName="/content/drive/MyDrive/Almacen_phd/FB13/sbert_test_mnr2"
embeddings_large_file="/content/drive/MyDrive/Almacen_phd/FB13/embedding_large.pt"
umap_embeddings_file="/content/drive/MyDrive/Almacen_phd/FB13/umap_embeddings.pt"
asignacion_clusters="/content/drive/MyDrive/Almacen_phd/FB13/clusters_labels"
os.chdir(pathFile)


# Preprocessing-Start

In [112]:
class InputExample(object):

    def __init__(self, guid, text_a, text_b=None, text_c=None, label=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.text_c = text_c
        self.label = label

In [113]:
class Triple(object):
    def __init__(self, guid, subject , predicate , obj, label):
        self.guid=guid
        self.subject=subject
        self.predicate=predicate
        self.obj=obj
        self.label=label

In [114]:
class ExampleTruncado(object):

    def __init__(self, text, label=None):
        self.text = text
        self.label = label

In [115]:
class DataProcessor(object):
  
    def get_train_examples(self, data_dir):
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        raise NotImplementedError()

    def get_labels(self, data_dir):
        raise NotImplementedError()

    def get_train_with_negatives(self,data_dir):
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            contador=0
            for line in reader:
                if contador>=100:
                    break
                contador=contador+1
                lines.append(line)
            return lines
            
    @classmethod
    def _read_train_with_negatives(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter=" ", quotechar=quotechar)
            lines = []
            for line in reader:         
                lines.append(line)
            return lines

In [116]:
class KGProcessor(DataProcessor):
    def __init__(self):
        self.labels = set()
    
    def get_train_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train", data_dir)
        
    def get_dev_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev", data_dir)

    def get_test_examples(self, data_dir):
        return self._create_examples(
          self._read_tsv(os.path.join(data_dir, "test_reduced.tsv")), "test", data_dir)
        
    def get_train_with_negatives(self,data_dir):
        return self._create_examples_with_negatives(
          self._read_train_with_negatives(os.path.join(data_dir, "train_reduced_with_negatives.txt")), "embeddings", data_dir)

    def get_relations(self, data_dir):
        with open(os.path.join(data_dir, "relations.txt"), 'r') as f:
            lines = f.readlines()
            relations = []
            for line in lines:
                relations.append(line.strip())
        return relations

    def get_labels(self, data_dir):
        return ["0", "1"]

    def get_entities(self, data_dir):
        with open(os.path.join(data_dir, "entities.txt"), 'r') as f:
            lines = f.readlines()
            entities = []
            for line in lines:
                entities.append(line.strip())
        return entities

    def get_prediction_test_triples(self, data_dir):
        with open(os.path.join(data_dir, "test_results.txt"), 'r') as f:
            lines = f.readlines()
            predictions = []
            for line in lines:               
                predictions.append(line.split()[2])
        return predictions

    def get_train_triples(self, data_dir):
        return self._read_train_with_negatives(os.path.join(data_dir, "train_reduced_with_negatives.txt"))    

    def get_dev_triples(self, data_dir):
        return self._read_tsv(os.path.join(data_dir, "dev.tsv"))

    def get_test_triples(self, data_dir):
        return self._read_tsv(os.path.join(data_dir, "test_reduced.tsv"))

    def _create_examples(self, lines, set_type, data_dir):
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in tqdm(ent_lines):
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    ent2text[temp[0]] = temp[1]

        entities = list(ent2text.keys())

        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r') as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')
                rel2text[temp[0]] = temp[1]      

        lines_str_set = set(['\t'.join(line) for line in lines])
        examples = []
        for (i, line) in enumerate(lines):
            auxiliar=["loke_joseph_of_austria_palatine_of_hungary","united_srena_gale", "archduke_joseph_of_austria_palatine_of_hungary"]
            flag_aux=False
            if line[0] or line[2] not in auxiliar:
                head_ent_text = ent2text[line[0]]
                tail_ent_text = ent2text[line[2]]
                relation_text = rel2text[line[1]]
                flag_aux=True
            else:
                head_ent_text = line[0]
                tail_ent_text = line[2]
                relation_text = line[1]
                flag_aux=True

            if set_type == "dev" or set_type == "test" and flag_aux:
                triple_label = line[3]
                if triple_label == "1":
                    label = "1"
                else:
                    label = "0"

                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text 
                self.labels.add(label)
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = text_c, label=label))
                
            elif set_type == "train" and flag_aux:
                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text 
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = text_c, label="1"))

                rnd = random.random()
                guid = "%s-%s" % (set_type + "_corrupt", i)
                if rnd <= 0.5:
                    # corrupting head
                    tmp_head = ''
                    while True:
                        tmp_ent_list = set(entities)
                        tmp_ent_list.remove(line[0])
                        tmp_ent_list = list(tmp_ent_list)
                        tmp_head = random.choice(tmp_ent_list)
                        tmp_triple_str = tmp_head + '\t' + line[1] + '\t' + line[2]
                        if tmp_triple_str not in lines_str_set:
                            break                    
                    tmp_head_text = ent2text[tmp_head]
                    examples.append(
                        InputExample(guid=guid, text_a=tmp_head_text, text_b=text_b, text_c = text_c, label="0"))       
                else:
                    # corrupting tail
                    tmp_tail = ''
                    while True:
                        tmp_ent_list = set(entities)
                        tmp_ent_list.remove(line[2])
                        tmp_ent_list = list(tmp_ent_list)
                        tmp_tail = random.choice(tmp_ent_list)
                        tmp_triple_str = line[0] + '\t' + line[1] + '\t' + tmp_tail
                        if tmp_triple_str not in lines_str_set:
                            break
                    tmp_tail_text = ent2text[tmp_tail]
                    examples.append(
                        InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = tmp_tail_text, label="0"))      
                                                            
        return examples

    def _create_examples_with_negatives(self, lines, set_type, data_dir):
        
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in tqdm(ent_lines):
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    ent2text[temp[0]] = temp[1]

        entities = list(ent2text.keys())

        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r') as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')
                rel2text[temp[0]] = temp[1]      

        examples = []
        for (i, line) in enumerate(lines):
            auxiliar=["loke_joseph_of_austria_palatine_of_hungary","united_srena_gale", "archduke_joseph_of_austria_palatine_of_hungary"]
            flag_aux=False
            if len(line)==5:
                if line[1] or line[3] not in auxiliar:               
                    guid=line[0]
                    head_ent_text = ent2text[line[1]]                
                    relation_text = rel2text[line[2]]
                    tail_ent_text = ent2text[line[3]]
                    triple_label = line[4]
                    examples.append(
                        InputExample(guid=guid, text_a=head_ent_text, text_b=relation_text, text_c = tail_ent_text, label=triple_label))     
                                                           
        return examples

In [117]:
def convert_examples_to_examplesTruncados(examples, max_seq_length, label_list, tokenizer=None, print_info = True):
    #examples=examples[:2000]
    label_map = {label : i for i, label in enumerate(label_list)}
    examples_truncados = []
    logger.info("")
    logger.info("Writing examples truncados")
    logger.info("max_seq_length %d" % (max_seq_length))
    separador="////////////// "

    for (ex_index, example) in tqdm(enumerate(examples)):
        
        tokens_a = example.text_a.split()
        tokens_b = None
        tokens_c = None
       
        if example.text_b and example.text_c:
            tokens_b = example.text_b.split()
            tokens_c = example.text_c.split()
           
            _truncate_seq_triple(tokens_a, tokens_b, tokens_c, max_seq_length)
        else:
            if len(tokens_a) > max_seq_length:
                tokens_a = tokens_a[:(max_seq_length)]
        tokens =tokens_a.copy()

        if tokens_b:
            tokens.extend(tokens_b)           
        if tokens_c:
            tokens.extend(tokens_c)        

        padding = ["0"] * (max_seq_length - len(tokens))
        tokens.extend(padding)
        assert len(tokens) == max_seq_length

        examples_truncados.append(
                ExampleTruncado(text=tokens,label=example.label))
    return examples_truncados

In [118]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        sujeto=tokens_a.split()
        relation=tokens_b.split()
        total_length = len(sujeto) + len(relation)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [119]:
def _truncate_seq_triple(tokens_a, tokens_b, tokens_c, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b) + len(tokens_c)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b) and len(tokens_a) > len(tokens_c):
            tokens_a.pop()
        elif len(tokens_b) > len(tokens_a) and len(tokens_b) > len(tokens_c):
            tokens_b.pop()
        elif len(tokens_c) > len(tokens_a) and len(tokens_c) > len(tokens_b):
            tokens_c.pop()
        else:
            tokens_c.pop()

# Preprocessing-End

In [120]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt = '%m/%d/%Y %H:%M:%S',
    level = logging.INFO)

processor = KGProcessor()  
label_list = processor.get_labels(pathFile)

In [121]:
#Test triples
ids_list=[]
subjets_list=[]
predicates_list=[]
objects_list=[]
labels_list=[]
_examples_test = processor.get_test_examples(pathFile) #se cargan las primeras 100 tripletas del conjunto test.tsv
for sample in _examples_test:
    ids_list.append(sample.guid)
    subjets_list.append(sample.text_a)
    predicates_list.append(sample.text_b)
    objects_list.append(sample.text_c)
    labels_list.append(sample.label)

data_ids=pd.Series(ids_list)
data_subjects=pd.Series(subjets_list)
data_predicates=pd.Series(predicates_list)
data_objects=pd.Series(objects_list)
data_labels=pd.Series(labels_list)

frame_triples_test=pd.DataFrame({'Id_Triple_Test':data_ids,'Subjects':data_subjects,'Predicates':data_predicates, "Objects":data_objects, "Label":data_labels})
kg_Bert_predictions_list=processor.get_prediction_test_triples(os.path.join(pathFile,"output_FB13" ))
frame_triples_test_reduced=frame_triples_test.iloc[:100]
frame_triples_test_reduced["KG BERT Prediction"]=kg_Bert_predictions_list[:100]
frame_triples_test_reduced

100%|██████████| 75042/75042 [00:00<00:00, 319288.61it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Id_Triple_Test,Subjects,Predicates,Objects,Label,KG BERT Prediction
0,test-0,Umberto I (Italian: Umberto Rainerio Carlo Ema...,cause of death,Tyrannicide is the killing or assassination of...,1,1
1,test-1,"John Glenn Beall Jr. (June 19, 1927 – March 24...",nationality,"The United States of America (USA), commonly k...",1,1
2,test-2,John Atkinson Grimshaw (6 September 1836 – 13 ...,gender,A male (♂) organism is the physiological sex t...,1,1
3,test-3,"Henry John Heinz (October 11, 1844 – May 14, 1...",nationality,"Germany (German: Deutschland, German pronuncia...",1,1
4,test-4,"Ernest Lundeen (August 4, 1878 – August 31, 19...",profession,A politician is a person active in party polit...,1,1
...,...,...,...,...,...,...
95,test-95,Giovanni Maria Lancisi (26 October 1654 – 20 J...,institution,"The University of Western Ontario (UWO), corpo...",0,0
96,test-96,"Mary Flannery O'Connor (March 25, 1925 – Augus...",religion,Anglicanism is a Western Christian tradition w...,0,0
97,test-97,"Daniel Webster (January 18, 1782 – October 24,...",profession,A war correspondent is a journalist who covers...,0,0
98,test-98,David Thomas Ansted FRS (5 February 1814 – 13 ...,profession,A mathematician is someone who uses an extensi...,0,1


In [122]:
#Train triples
ids_list2=[]
subjets_list2=[]
predicates_list2=[]
objects_list2=[]
labels_list2=[]
_examples_train=processor.get_train_with_negatives(pathFile) 
for sample2 in _examples_train:
    ids_list2.append(sample2.guid)
    subjets_list2.append(sample2.text_a)
    predicates_list2.append(sample2.text_b)
    objects_list2.append(sample2.text_c)
    labels_list2.append(sample2.label)

data2_ids=pd.Series(ids_list2)
data2_subjects=pd.Series(subjets_list2)
data2_predicates=pd.Series(predicates_list2)
data2_objects=pd.Series(objects_list2)
data2_labels=pd.Series(labels_list2)

frame_triples_test=pd.DataFrame({'Id_Triple_Training':data2_ids, 'Subjects':data2_subjects,'Predicates':data2_predicates, "Objects":data2_objects, "Label":data2_labels})
frame_triples_test=frame_triples_test[:100]
frame_triples_test

100%|██████████| 75042/75042 [00:00<00:00, 285916.82it/s]


,Id_Triple_Training,Subjects,Predicates,Objects,Label
0,train-0,"Denys Arthur Rayner DSC & Bar, VRD, RNVR (9 Fe...",cause of death,Cancer is a group of diseases involving abnorm...,1
1,train_corrupt-0,"Denys Arthur Rayner DSC & Bar, VRD, RNVR (9 Fe...",cause of death,Thomas Langlois Lefroy (8 January 1776 – 4 May...,0
2,train-1,Karl Wilhelm Otto Lilienthal (23 May 1848 – 10...,cause of death,"In aviation, an accident is defined by the Con...",1
3,train_corrupt-1,Karl Wilhelm Otto Lilienthal (23 May 1848 – 10...,cause of death,"John Manners, 2nd Duke of Rutland KG (18 Septe...",0
4,train-2,Rajiv Ratna Gandhi ( (listen); 20 August 1944 ...,cause of death,Assassination is the act of killing a prominen...,1
...,...,...,...,...,...
95,train_corrupt-47,Grigoris Pieris Afxentiou (Greek: Γρηγόρης Πιε...,nationality,"Armenia ( (listen); Armenian: Հայաստան, romani...",0
96,train-48,"François-Vincent Raspail, L.L.D., M.D. (25 Jan...",nationality,"France (French: [fʁɑ̃s]), officially the Frenc...",1
97,train_corrupt-48,"Michael Anthony Fleming, O.S.F. (c. 1792 – Jul...",nationality,"France (French: [fʁɑ̃s]), officially the Frenc...",0
98,train-49,Hungry Joe,nationality,"The United States of America (USA), commonly k...",1


In [123]:
id_input_triple=input("Enter the test triple id to be explained:")
frame_triples_test_reduced[frame_triples_test_reduced["Id_Triple_Test"]==id_input_triple]

Enter the test triple id to be explained:test-99


,Id_Triple_Test,Subjects,Predicates,Objects,Label,KG BERT Prediction
99,test-99,"Mahlon R. Pitney (February 5, 1858 – December ...",nationality,The Central African Republic (CAR; Sango: Ködö...,0,0


In [124]:
#Turn training triples + target test triple into sequences 
_examples_train_truncates= convert_examples_to_examplesTruncados(_examples_train[:100], 512,label_list)
X_train=[]
for sample in _examples_train_truncates:
    X_train.append(" ".join(sample.text))

target_triple_test=list(e for e in _examples_test if e.guid  == id_input_triple)
target_triple_sequence_truncate=convert_examples_to_examplesTruncados(target_triple_test, 512,label_list)
X_train.append(" ".join(target_triple_sequence_truncate[0].text))

06/23/2022 05:21:48 - INFO - __main__ -   
06/23/2022 05:21:48 - INFO - __main__ -   Writing examples truncados
06/23/2022 05:21:48 - INFO - __main__ -   max_seq_length 512
100it [00:00, 3453.32it/s]
06/23/2022 05:21:48 - INFO - __main__ -   
06/23/2022 05:21:48 - INFO - __main__ -   Writing examples truncados
06/23/2022 05:21:48 - INFO - __main__ -   max_seq_length 512
1it [00:00, 5370.43it/s]


In [125]:
#Get the sequences embeddings
model = SentenceTransformer(modelName)
embeddings_large = model.encode(X_train, show_progress_bar=True) 
torch.save(embeddings_large,embeddings_large_file) 

06/23/2022 05:21:48 - INFO - sentence_transformers.SentenceTransformer -   Load pretrained SentenceTransformer: /content/drive/MyDrive/Almacen_phd/FB13/sbert_test_mnr2
06/23/2022 05:21:52 - INFO - sentence_transformers.SentenceTransformer -   Use pytorch device: cuda


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [126]:
#Dimension reduction with UMAP
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5,
                            min_dist=0.0,
                            metric='cosine').fit_transform(embeddings_large)
torch.save(umap_embeddings, umap_embeddings_file)
print(umap_embeddings.shape)
pd.DataFrame(umap_embeddings)

(101, 5)


,0,1,2,3,4
0,3.139486,8.501579,10.313164,10.273165,8.115463
1,18.869400,-0.773885,12.145260,6.422682,5.137289
2,18.493719,-0.146690,11.979384,6.818827,5.127192
3,18.784159,-0.517393,12.141944,6.299152,5.334672
4,2.243906,8.804002,9.797422,9.595801,7.551284
...,...,...,...,...,...
96,-0.244470,10.512479,7.684050,7.443296,5.809157
97,-0.405709,10.703627,7.613851,7.316225,5.804186
98,-1.243424,10.647042,7.822615,7.614900,5.427052
99,-1.366198,10.528052,7.980968,7.582489,5.352841


In [127]:
#To cluster embeddings with HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size=5, 
                        metric='euclidean', 
                        cluster_selection_method='eom'
                        ).fit(umap_embeddings)

#Saving each triple with its cluster label
with open(asignacion_clusters, "w") as writer:
    writer.write("Clusters asignados\n")
    writer.write("id de tripleta -------cluster\n")
    for id, idcluster in enumerate(cluster.labels_):
        writer.write("%d ---------- %d\n" % (id, idcluster))    

print(f"Number of clusters = {np.unique(cluster.labels_).shape[0] - 1}") 

Number of clusters = 4


In [128]:
#Getting the positions of triples in the same cluster of the target triple
cluster_target=cluster.labels_[len(cluster.labels_)-1]
position_triples_targets=[pos for pos,value in enumerate(cluster.labels_) if value==cluster_target]

In [129]:
#Target triple set
target_triples_set = [example for pos, example in enumerate(_examples_train[:100]) if pos in position_triples_targets]
ids_list3=[]
subjets_list3=[]
predicates_list3=[]
objects_list3=[]
labels_list3=[]
for sample3 in target_triples_set:
    ids_list3.append(sample3.guid)
    subjets_list3.append(sample3.text_a)
    predicates_list3.append(sample3.text_b)
    objects_list3.append(sample3.text_c)
    labels_list3.append(sample3.label)

data3_ids=pd.Series(ids_list3)
data3_subjects=pd.Series(subjets_list3)
data3_predicates=pd.Series(predicates_list3)
data3_objects=pd.Series(objects_list3)
data3_labels=pd.Series(labels_list3)
frame3_target_triple_train=pd.DataFrame({'Id_triple_Training':data3_ids, 'Subjects':data3_subjects,'Predicates':data3_predicates, "Objects":data3_objects, "Label":data3_labels})
frame3_target_triple_train

,Id_triple_Training,Subjects,Predicates,Objects,Label
0,train_corrupt-22,"Jim Gary (March 17, 1939 – January 14, 2006) w...",cause of death,Prince Ludwig of Hesse and by Rhine (Ludwig Er...,0
1,train-31,Mary de Bohun (c. 1369/70 – 4 June 1394) was t...,nationality,The Kingdom of England (Anglo-Norman and Frenc...,1
2,train_corrupt-31,Robert Whitehead (3 January 1823 – 14 November...,nationality,The Kingdom of England (Anglo-Norman and Frenc...,0
3,train-32,"Roger Atkinson Pryor (July 19, 1828 – March 14...",nationality,"The United States of America (USA), commonly k...",1
4,train_corrupt-32,Giambattista Bodoni (Italian pronunciation: [d...,nationality,"The United States of America (USA), commonly k...",0
5,train-33,"Thomas J. Tedesco (July 3, 1930 – November 10,...",nationality,"The United States of America (USA), commonly k...",1
6,train-34,"Count Gotō Shinpei (後藤 新平, 24 July 1857 – 13 A...",nationality,Japan (Japanese: 日本; Nippon [ɲippoɴ] or Nihon ...,1
7,train-35,Antonio Cornejo-Polar (1936–1997) was a Peruvi...,nationality,Peru ( (listen); Spanish: Perú [peˈɾu]; Quechu...,1
8,train-36,"Joseph Henry Bottum (August 7, 1903 – July 4, ...",nationality,"The United States of America (USA), commonly k...",1
9,train_corrupt-36,"Robert Edward Coontz (June 11, 1864 – January ...",nationality,"The United States of America (USA), commonly k...",0


In [130]:
#Target triple set embeddings
umap_embeddings_target_triple_set= [vector for pos, vector in enumerate(umap_embeddings[:-1]) if pos in position_triples_targets]
#umap_embeddings_target_triple_set_np=np.array(umap_embeddings_target_triple_set)
umap_embeddings_target_triple_set_df=pd.DataFrame(umap_embeddings_target_triple_set)

ids_triple_target_set= [sample.guid for pos, sample in enumerate(_examples_train[:100]) if pos in position_triples_targets]
labels_triple_target_set= [sample.label for pos, sample in enumerate(_examples_train[:100]) if pos in position_triples_targets]

umap_embeddings_target_triple_set_df["Id_triple_Training"]=ids_triple_target_set
umap_embeddings_target_triple_set_df["Label"]=labels_triple_target_set

umap_embeddings_target_triple_set_df.head() 

umap_embeddings_target_triple_set_df.to_excel("Embeddings_Regresion.xlsx")
umap_embeddings_target_triple_set_df

,0,1,2,3,4,Id_triple_Training,Label
0,0.564651,9.971502,8.440087,7.911705,6.066451,train_corrupt-22,0
1,-0.665915,10.385058,7.950857,7.244218,5.270862,train-31,1
2,-0.574219,10.321492,7.998509,7.288590,5.208165,train_corrupt-31,0
3,-1.457406,10.373721,7.968051,7.628969,5.237294,train-32,1
4,0.278064,10.136518,8.313503,7.758735,5.993969,train_corrupt-32,0
5,-1.484123,10.375258,7.831549,7.922109,5.440887,train-33,1
6,-1.521299,10.499785,7.802799,7.697378,5.464177,train-34,1
7,-0.876526,10.826502,8.181039,7.414748,5.669445,train-35,1
8,-1.257045,10.472176,7.797880,7.707376,5.352361,train-36,1
9,-1.302493,10.323926,7.961373,7.545544,5.524577,train_corrupt-36,0


In [131]:
#Logistic Regression Prediction
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(umap_embeddings_target_triple_set_df.iloc[:,:5],umap_embeddings_target_triple_set_df.iloc[:,6]) 

pred = lr.predict(umap_embeddings[-1,:].reshape(1,5))
pred
log_regresion=pred[0]
log_regresion

'1'

In [132]:
#Decision Tree Prediction
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

dt = DecisionTreeClassifier(max_depth=10)
dt.fit(umap_embeddings_target_triple_set_df.iloc[:,:5],umap_embeddings_target_triple_set_df.iloc[:,6])
pred = dt.predict(umap_embeddings[-1,:].reshape(1,5))
arbol_decision=pred[0]
arbol_decision

'1'

In [133]:
#Representar arbol en texto plano
from sklearn.tree import export_text
r = export_text(dt)
print(r)

|--- feature_1 <= 10.34
|   |--- feature_0 <= 0.20
|   |   |--- feature_1 <= 10.30
|   |   |   |--- class: 1
|   |   |--- feature_1 >  10.30
|   |   |   |--- class: 0
|   |--- feature_0 >  0.20
|   |   |--- class: 0
|--- feature_1 >  10.34
|   |--- feature_1 <= 10.65
|   |   |--- feature_2 <= 7.97
|   |   |   |--- class: 1
|   |   |--- feature_2 >  7.97
|   |   |   |--- feature_2 <= 8.02
|   |   |   |   |--- class: 0
|   |   |   |--- feature_2 >  8.02
|   |   |   |   |--- class: 1
|   |--- feature_1 >  10.65
|   |   |--- feature_1 <= 10.75
|   |   |   |--- feature_3 <= 7.87
|   |   |   |   |--- class: 0
|   |   |   |--- feature_3 >  7.87
|   |   |   |   |--- class: 1
|   |   |--- feature_1 >  10.75
|   |   |   |--- class: 1



In [134]:
from numpy.core.fromnumeric import squeeze
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
knc = KNeighborsClassifier(n_neighbors=5) 
rnc = RadiusNeighborsClassifier(radius=5) 
knc.fit(umap_embeddings_target_triple_set_df.iloc[:,:5],umap_embeddings_target_triple_set_df.iloc[:,6])
rnc.fit(umap_embeddings_target_triple_set_df.iloc[:,:5],umap_embeddings_target_triple_set_df.iloc[:,6])
knc_pred = knc.predict(umap_embeddings[-1,:].reshape(1,5))
knc.kneighbors(umap_embeddings[-1,:].reshape(1,5))
_, knc_neighbors = knc.kneighbors(umap_embeddings[-1,:].reshape(1,5))
knn_prediction=knc_pred[0]
vecinos=knc_neighbors.squeeze().tolist()
nearest_neighbors_df=umap_embeddings_target_triple_set_df.iloc[vecinos]
nearest_neighbors_df

,0,1,2,3,4,Id_triple_Training,Label
21,-0.455795,10.734657,7.823128,7.358394,5.808185,train_corrupt-44,0
14,-0.551217,10.629705,7.637064,7.269383,5.855266,train-40,1
28,-0.405709,10.703627,7.613851,7.316225,5.804186,train_corrupt-48,0
15,-0.346189,10.752862,7.680582,7.538823,6.019085,train-41,1
17,-0.833061,10.750943,7.812007,7.530333,5.536211,train_corrupt-42,0


In [135]:
from numpy.ma.core import append
#tripletas en version corta por los ids de los vecinos y tripleta target
id_tripletas=nearest_neighbors_df.iloc[:,5].unique()
#En este momento hay 200 tripletas en le fichero train_with_negatives.txt.
train_triples_sequences=processor.get_train_triples(pathFile)
triples=[]
triples_vecinos= [triple for triple in train_triples_sequences if triple[0] in id_tripletas]
for a in triples_vecinos:
    triples.append(Triple(guid=a[0],subject=a[1],predicate=a[2],obj=a[3], label=a[4]))

id=int(id_input_triple.split("-")[1])
#print(id)
tripletarget_sindescripcion=processor.get_test_triples(pathFile)[id]
print('\033[1m'+"Triple target \033[0m:  {}          {}         {}".format(tripletarget_sindescripcion[0], tripletarget_sindescripcion[1],tripletarget_sindescripcion[2]))
#print("Triple target: Subject %s   Predicate %s   Object %s " % tripletarget_sindescripcion[0], tripletarget_sindescripcion[1],tripletarget_sindescripcion[2])

ids_list4=[]
subjets_list4=[]
predicates_list4=[]
objects_list4=[]
labels_list4=[]

for sample4 in triples:
    ids_list4.append(sample4.guid)
    subjets_list4.append(sample4.subject)
    predicates_list4.append(sample4.predicate)
    objects_list4.append(sample4.obj)
    labels_list4.append(sample4.label)

data4_ids=pd.Series(ids_list4)
data4_subjects=pd.Series(subjets_list4)
data4_predicates=pd.Series(predicates_list4)
data4_objects=pd.Series(objects_list4)
data4_labels=pd.Series(labels_list4)

frame4_target_triple_train=pd.DataFrame({'Id_triple_Training':data4_ids, 'Subjects':data4_subjects,'Predicates':data4_predicates, "Objects":data4_objects, "Label":data4_labels})
frame4_target_triple_train


#****mostrar las componentes de la tripleta.


#Tripleta target y tripletas reducidas
#Tripletas target y ejemplos ampliados
#[[sample.text_a, sample2.text_b, sample2.text_c, sample.label] for sample in _examples_train if sample.guid in id_tripletas]
#[[sample.text_a, sample2.text_b, sample2.text_c, sample.label] for sample in _examples_train if sample.guid in id_tripletas]


Triple target :  mahlon_pitney          nationality         central_african_republic


,Id_triple_Training,Subjects,Predicates,Objects,Label
0,train-40,charles_berny_douville,nationality,france,1
1,train-41,ambrogio_bergognone,nationality,italy,1
2,train_corrupt-42,james_camp_tappan,nationality,germany,0
3,train_corrupt-44,enrique_penaranda,nationality,france,0
4,train_corrupt-48,michael_anthony_fleming,nationality,france,0
